In [ ]:
%pip install mediapipe-model-maker
%pip install --upgrade jupyter ipywidgets

In [ ]:
import os
import tensorflow as tf

assert tf.__version__.startswith("2")

from mediapipe_model_maker import gesture_recognizer

In [ ]:
# Remove temporary model maker files because they mess with the training
!rm -rf /tmp/model_maker
!rm -rf ./out

In [ ]:
DATASET_PATH = "dataset/pictures"

labels = []
for i in os.listdir(DATASET_PATH):
    if os.path.isdir(os.path.join(DATASET_PATH, i)):
        labels.append(i)
print("Labels:", labels)

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=DATASET_PATH, hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="out", epochs=120)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data, validation_data=validation_data, options=options
)

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

In [ ]:
model.export_model()

# Move the model to the app's assets directory
!mv out/gesture_recognizer.task app/src/main/assets/szj.task